In [5]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [6]:
#CPCG0183

In [7]:
#Read the data

GenomicFeatures_CPCG0183 = pd.read_csv(open('data/CPCG0183/GenomicFeatures_CPCG0183.txt','r'),sep='\t')
SNVCalls_CPCG0183    = pd.read_csv(open('data/CPCG0183/SNVCalls_CPCG0183.txt','r'),sep='\t') 

In [8]:
print GenomicFeatures_CPCG0183.shape
print SNVCalls_CPCG0183.shape

(143223, 15)
(143223, 99)


In [9]:
GenomicFeatures_CPCG0183.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,16257,180,14,5390,210,98,23,58.5,CCC,2.333333,0.517413,ncRNA_intronic,35222,0.638095
1,chr1,16957,104,15,3487,159,63,0,45.0,TCA,2.443662,0.601990,NaN,34522,0.459119
2,chr1,17538,119,16,3462,140,79,37,55.0,GCC,2.719424,0.676617,ncRNA_intronic,33941,0.442857
3,chr1,20144,161,32,5296,202,88,23,45.0,GCC,2.394558,0.487562,ncRNA_intronic,31335,0.599010
4,chr1,20467,89,44,3975,150,85,0,50.5,GCT,2.717391,0.477612,NaN,31012,0.466667


In [10]:
SNVCalls_CPCG0183.head()

,CHROM,POS,END,X2399003,X2642551,X2642553,X2673210,X2673224,X2679044,X2679047,X2706565,X2791038,X2811015,X2811019,X2811021,X2811024,X2811385,X2812902,X2813241,X2813273,
0,chr1,16256,16257,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,chr1,16956,16957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,17537,17538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...
3,chr1,20143,20144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...
4,chr1,20466,20467,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...


In [11]:
np.unique(GenomicFeatures_CPCG0183['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'NNG', 'TCA', 'TCC',
       'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [12]:
CPCG0183 = pd.merge(GenomicFeatures_CPCG0183, SNVCalls_CPCG0183,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [13]:
CPCG0183.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399003,X2642551,X2642553,
0,chr1,16257,180,14,5390,210,98,23,58.5,CCC,2.333333,0.517413,ncRNA_intronic,35222,0.638095,16256,16257,1,0,0,...
1,chr1,16957,104,15,3487,159,63,0,45.0,TCA,2.443662,0.601990,NaN,34522,0.459119,16956,16957,0,0,0,...
2,chr1,17538,119,16,3462,140,79,37,55.0,GCC,2.719424,0.676617,ncRNA_intronic,33941,0.442857,17537,17538,0,0,0,...
3,chr1,20144,161,32,5296,202,88,23,45.0,GCC,2.394558,0.487562,ncRNA_intronic,31335,0.599010,20143,20144,0,0,0,...
4,chr1,20467,89,44,3975,150,85,0,50.5,GCT,2.717391,0.477612,NaN,31012,0.466667,20466,20467,0,0,0,...


In [14]:
CPCG0183_X = CPCG0183.iloc[:,2:15] #Exclude CHROM, POS
CPCG0183_Y = CPCG0183.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [15]:
print CPCG0183_X.shape
print CPCG0183_Y.shape

(143223, 13)
(143223, 96)


In [16]:
CPCG0183_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,180,14,5390,210,98,23,58.5,CCC,2.333333,0.517413,ncRNA_intronic,35222,0.638095
1,104,15,3487,159,63,0,45.0,TCA,2.443662,0.601990,NaN,34522,0.459119


In [17]:
CPCG0183_Y[:2]

,X2399003,X2642551,X2642553,X2673210,X2673224,X2679044,X2679047,X2706565,X2791038,X2811015,X2811019,X2811021,X2811024,X2811385,X2812902,X2813241,X2813273,X2813275,X2813339,X2813370,
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [18]:
totalSample = CPCG0183_Y.shape[0]
CPCG0183_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print CPCG0183_Y_Val.shape

143223
(143223, 1)


In [19]:
CPCG0183_Y.shape

(143223, 96)

In [20]:
CPCG0183_Y.values[0].shape

(96,)

In [21]:
for i in range(totalSample):
    CPCG0183_Y_Val[i] = np.argmax(np.bincount(CPCG0183_Y.values[i]))

In [22]:
CPCG0183_Y_Val_1 = pd.DataFrame(CPCG0183_Y_Val)
CPCG0183_Y_Val_1.columns = ['Label']

In [23]:
CPCG0183_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [24]:
CPCG0183_dataframe = pd.concat([CPCG0183_X, CPCG0183_Y_Val_1],axis=1)

In [25]:
CPCG0183_dataframe.shape

(143223, 14)

In [26]:
CPCG0183_dataframe.to_csv('CPCG0183.csv')

In [29]:
#Now we have variable importance ...lets do analysis

In [30]:
%load_ext rpy2.ipython

In [32]:
%%R
# Read in the data
CPCG0183Frame = read.csv("CPCG0183.csv")
#str(CPCG0183Frame)

'data.frame':	143223 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  180 104 119 161 89 26 50 45 61 48 ...
 $ NonRefAllele   : int  14 15 16 32 44 5 14 20 25 39 ...
 $ BaseQual       : int  5390 3487 3462 5296 3975 939 1900 1923 2516 2700 ...
 $ TumourCoverage : int  210 159 140 202 150 33 66 73 98 94 ...
 $ NormalCoverage : int  98 63 79 88 85 28 37 46 56 61 ...
 $ MapQual        : num  23 0 37 23 0 46 60 15 13.5 29 ...
 $ ReadPosition   : num  58.5 45 55 45 50.5 42 57 52 49 51.5 ...
 $ Trinucleotide  : Factor w/ 33 levels "ACA","ACC","ACG",..: 10 26 18 18 20 13 1 5 27 13 ...
 $ HomopolymerRate: num  2.33 2.44 2.72 2.39 2.72 ...
 $ GCcontent      : num  0.517 0.602 0.677 0.488 0.478 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 8 1 8 8 1 5 1 5 1 5 ...
 $ DistanceSNP    : int  35222 34522 33941 31335 31012 2348 2470 2847 2576 2626 ...
 $ StrandBias     : num  0.638 0.459 0.443 0.599 0.467 ...
 $ Label          : num  0 

In [35]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
sum(is.na(CPCG0183Frame$ReadPosition)) 

[1] 30


In [36]:
%%R
CPCG0183Frame$ReadPosition[is.na(CPCG0183Frame$ReadPosition)] = mean(CPCG0183Frame$ReadPosition, na.rm=TRUE)
CPCG0183Frame$StrandBias[is.na(CPCG0183Frame$StrandBias)] = mean(CPCG0183Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0183Frame$ReadPosition)) )
print (sum(is.na(CPCG0183Frame$StrandBias)) )

[1] 0
[1] 0


In [37]:
%%R
sum(is.na(CPCG0183Frame))  #No NA value now

[1] 0


In [38]:
%%R
write.csv(CPCG0183Frame, file = "CPCG0183_imputed.csv")  #Cleaned CSV without NA

In [39]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpDmw9xo/h2o_rbhat_started_from_r.out
    /tmp/RtmpDmw9xo/h2o_rbhat_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 seconds 943 milliseconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_szm488 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.60 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 

Note:  As started, H2O is limited to the CRAN default of 2 CPUs.
       Shut down and restart H2O as shown below to use all your CPUs.
           > h2o.s

In [40]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/CPCG0183_imputed.csv"
CPCG0183.hex          = h2o.importFile(path = csvPath, destination_frame = "CPCG0183.hex") 
train                 = as.data.frame(CPCG0183.hex)

#train  = subset(train,C1<20000)

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%
  |======================================================================| 100%


In [41]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [44]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
           variable relative_importance scaled_importance percentage
1      NonRefAllele            1.000000          1.000000   0.431964
2    NormalCoverage            0.128271          0.128271   0.055408
3    TumourCoverage            0.108174          0.108174   0.046727
4 Trinucleotide.NNG            0.055045          0.055045   0.023778
5         RefAllele            0.053923          0.053923   0.023293

---
                     variable relative_importance scaled_importance percentage
56     GenomicLocation.exonic            0.005418          0.005418   0.002340
57   GenomicLocation.intronic            0.004980          0.004980   0.002151
58                 StrandBias            0.004915          0.004915   0.002123
59          Trinucleotide.CCT            0.004698          0.004698   0.002029
60 GenomicLocation.intergenic            0.001672          0.001672   0.000722
61  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [45]:
%%R
write.table(var_importance, file="CPCG0183_variable_importance.csv")

In [46]:
var_importance = pd.read_csv(open('CPCG0183_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [47]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.431964
2,NormalCoverage,0.128271,0.128271,0.055408
3,TumourCoverage,0.108174,0.108174,0.046727
4,Trinucleotide.NNG,0.055045,0.055045,0.023778
5,RefAllele,0.053923,0.053923,0.023293


In [48]:
GenomicFeatures_CPCG0183[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,16257,180,14,5390,210,98,23,58.5,CCC,2.333333,0.517413,ncRNA_intronic,35222,0.638095
1,chr1,16957,104,15,3487,159,63,0,45.0,TCA,2.443662,0.601990,NaN,34522,0.459119
2,chr1,17538,119,16,3462,140,79,37,55.0,GCC,2.719424,0.676617,ncRNA_intronic,33941,0.442857
3,chr1,20144,161,32,5296,202,88,23,45.0,GCC,2.394558,0.487562,ncRNA_intronic,31335,0.599010
4,chr1,20467,89,44,3975,150,85,0,50.5,GCT,2.717391,0.477612,NaN,31012,0.466667


In [49]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.045852318406105007,
 'DistanceSNP': 0.025250218808651002,
 'GCcontent': 0.0087331701070070301,
 'GenomicLocation.UTR3': 0.021850556135177598,
 'GenomicLocation.UTR5': 0.030610755085945102,
 'GenomicLocation.downstream': 0.021469132974743801,
 'GenomicLocation.exonic': 0.00541807897388935,
 'GenomicLocation.exonic;splicing': 0.022779917344451003,
 'GenomicLocation.intergenic': 0.0016719751292839599,
 'GenomicLocation.intronic': 0.00498037366196513,
 'GenomicLocation.missing(NA)': 0.0150138847529888,
 'GenomicLocation.ncRNA_UTR3': 0.041083738207817098,
 'GenomicLocation.ncRNA_UTR5': 0.036083277314901401,
 'GenomicLocation.ncRNA_exonic': 0.0293677560985088,
 'GenomicLocation.ncRNA_intronic': 0.0089321816340088792,
 'GenomicLocation.ncRNA_splicing': 0.052761055529117598,
 'GenomicLocation.splicing': 0.031691562384366996,
 'GenomicLocation.upstream': 0.013549567200243501,
 'GenomicLocation.upstream;downstream': 0.049730904400348698,
 'HomopolymerRate': 0.0145589150488377,
 'M

In [51]:
noOfSample = GenomicFeatures_CPCG0183.shape[0]

CPCG0183_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_CPCG0183.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_CPCG0183.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_CPCG0183.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_CPCG0183.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_CPCG0183.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_CPCG0183.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_CPCG0183.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_CPCG0183.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_CPCG0183.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_CPCG0183.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_CPCG0183.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_CPCG0183.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_CPCG0183.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_CPCG0183.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_CPCG0183.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_CPCG0183.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' DistanceSNP=',DistanceSNP
    #print 'StrandBias=',StrandBias
    CPCG0183_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [52]:
#Sorting dictionary
CPCG0183_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in CPCG0183_Dict.items()],reverse=True)]

In [53]:
CPCG0183_Dict_Frame = pd.DataFrame(CPCG0183_Dict1, columns=['POS', 'Value'])

In [54]:
CPCG0183_Dict_Frame.to_csv("CPCG0183_Dict_Frame.csv")

In [27]:
CPCG0183_Dict_Frame = pd.read_csv("CPCG0183_Dict_Frame.csv")

In [28]:
GenomicFeatures_CPCG0183_1 = pd.concat([CPCG0183.iloc[:,0:16], CPCG0183_Y_Val_1],axis=1)

In [34]:
rows = CPCG0183_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_CPCG0183_Top20 = GenomicFeatures_CPCG0183_1.loc[GenomicFeatures_CPCG0183_1['POS_x'].isin(rows)]

[ 27732058 121484599    158331    213981    329418    372541    456182
    616357    873789    875303    932444   1260170   1260862   1317293
   1317287   1354755   1370771   1413967  42385695   1578960]


In [36]:
GenomicFeatures_CPCG0183_Top20[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
5461,chr1,121484599,4612,3466,210533,279474,228302,29,47.0,TCT,2.597222,0.393035,NaN,8363,0.544301,121484598,0
14977,chr10,42385695,7713,342,247425,66279,87595,15,75.0,TCA,2.359155,0.343284,NaN,11642,0.362106,42385694,0
56831,chr19,27732058,5898,2175,243874,369873,323774,17,53.0,ATA,2.261745,0.383085,NaN,8652,0.546469,27732057,0
134414,chrX,158331,48,2,1534,60,23,0,69.5,ACC,1.684848,0.482587,intergenic,2541224,0.500000,158330,0
134417,chrX,213981,6,2,243,10,8,0,55.0,ATC,2.548611,0.472637,intronic,2485574,0.500000,213980,0
134421,chrX,329418,3,3,181,8,7,0,48.0,CCG,5.000000,0.776119,intronic,2370137,0.750000,329417,0
134425,chrX,372541,18,0,390,24,12,0,28.0,GCG,2.634483,0.577114,intergenic,2327014,0.541667,372540,0
134428,chrX,456182,42,1,1075,44,26,0,58.0,TTG,2.881119,0.432836,intergenic,2243373,0.659091,456181,0
134433,chrX,616357,4,1,0,7,1,0,67.0,ATA,1.951220,0.378109,intronic,2083198,0.285714,616356,0
134440,chrX,873789,42,2,1301,54,31,0,66.0,ATT,2.775362,0.393035,intergenic,1825766,0.462963,873788,0


In [ ]:

GenomicFeatures_CPCG0183_Top20.to_csv('GenomicFeatures_CPCG0183_Top20.csv')